# NearestNeighbors Demo
This demo illustrates how SIMBSIG can be used for Nearest Neighbors queries, and how the use compares to scikit-learn. 

In [1]:
!pip install simbsig

In [5]:
from simbsig.neighbors import NearestNeighbors
from sklearn.neighbors import NearestNeighbors as NearestNeighbors_sk
import h5py as h5py
import numpy as np
import os

## Set Parameters

In [6]:
n_train_samples = 10
n_query_samples = 5
n_dim = 2

## Create Toy Data
### numpy arrays

In [7]:
# numpy arrays
np.random.seed(98)
X_train = np.random.uniform(low=-5, high=5, size=(n_train_samples, n_dim))
X_query = np.random.uniform(low=-5, high=5, size=(n_query_samples, n_dim))


### hdf5 files
#### Google Colab
If you work from google colab, you can execute the statement below: Of course you can also store the data in a directory more suitable to you.

In [8]:
# when working on google colab, google drive can be used to save and read data
from google.colab import drive
drive.mount('/content/drive')

# depending on the structure of your google drive, you might want to choose a different dataset path
dataset_path = '/content/drive/MyDrive/'

ModuleNotFoundError: No module named 'google'

#### Your Computer
Alternatively, if you work from your computer, you can execute the statement below:
Of course you can also store the data in a directory more suitable to you.

In [ ]:
# when working from your computer, your disk can be used to save and read data
import os
dataset_path = os.path.dirname(os.path.realpath("__file__"))

In [ ]:
# hdf5 files using h5py
train_file = f'train.hdf5'
query_file = f'query.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

## Scikit-Learn


In [20]:
nn_sk = NearestNeighbors_sk(n_neighbors=2)

nn_sk.fit(X_train)
nn_sk.kneighbors(X_query, return_distance=False)

array([[6, 2],
       [9, 0],
       [4, 5],
       [1, 8],
       [0, 9]])

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG.


In [21]:
nn = NearestNeighbors(n_neighbors=2)

nn.fit(X_train)
nn.kneighbors(X_query, return_distance=False)

X_train for X_query progress:   0%|          | 0/1 [00:00<?, ?it/s]


array([[6, 2],
       [9, 0],
       [4, 5],
       [1, 8],
       [0, 9]])

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `hdf5`

In [23]:
nn_hdf5 = NearestNeighbors(n_neighbors=2, mode='hdf5')

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
res = nn_hdf5.kneighbors(query_data, return_distance=False)

# close hdf5 file
train_data.close()
query_data.close()

res

X_train for X_query progress:   0%|          | 0/1 [00:00<?, ?it/s]


array([[6, 2],
       [9, 0],
       [4, 5],
       [1, 8],
       [0, 9]])

### Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs, numpy arrays and hdf5 files. If you are working on google colab, remember to  enable GPU as hardware accelerator under Runtime.

In [24]:
nn_hdf5 = NearestNeighbors(n_neighbors=2, mode='hdf5', device='gpu')

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
res = nn_hdf5.kneighbors(query_data, return_distance=False)

# close hdf5 file
train_data.close()
query_data.close()

res

AssertionError: Torch not compiled with CUDA enabled